In [1]:
import mysql.connector
cnx = mysql.connector.connect(user='root', password='9838', host='mariadb', database="db_spark")

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession \
    .builder \
    .appName("Python Spark SQL basic example") \
    .getOrCreate()

# 9. 데이터 소스

## 9.1 데이터소스 API의 구조

#### 읽기 모드
+ Permissive: 오류 레코드의 모든 필드를 null로 설정하고 모든 오류 레코드를 \_corrup_record 라는 문자열 컬럼에 기록
+ dropMalfromed: 형식에 맞지 않는 레코드가 포함된 로우 제거
+ failFast: 형식에 맞지 않는 레코드를 만나면 즉시 종료

#### 쓰기 모드
+ append: 해당 경로에 이미 존재하는 파일 몰록에 결과 파일을 추가
+ overwrite: 이미 존재하는 모든 데이터를 완전히 덮어 씀
+ errorIfExists: 해당 경로에 데이터나 파일이 존재하는 경우 오류를 발생시키면서 쓰기 작업이 실패됨
+ ignore: 해당 경로에 데이터나 파일이 존재하는 경우 아무런 처리도 하지 않음


## 9.2 CSV

### 9.2.1 CSV 파일 쓰기

In [3]:
from pyspark.sql.types import StructField, StructType, StringType, LongType 

myManualSchema = StructType([
    StructField("DEST_COUNTRY_NAME", StringType(), True),
    StructField("ORIGIN_COUNTRY_NAME", StringType(), True),
    StructField("count", LongType(), False)
])

csvFile = spark.read.format("csv")\
    .option("header", "true")\
    .option("mode", "permissive")\
    .schema(myManualSchema)\
    .load("./data/flight-data/csv/2010-summary.csv")\

In [4]:
csvFile.show(5)

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|    United States|            Romania|    1|
|    United States|            Ireland|  264|
|    United States|              India|   69|
|            Egypt|      United States|   24|
|Equatorial Guinea|      United States|    1|
+-----------------+-------------------+-----+
only showing top 5 rows



### 9.2.3 CSV 파일 쓰기

In [5]:
csvFile.write.format("csv").mode("overwrite")\
    .option("sep", "\t")\
    .save("./data/tmp/my-tsv-file.tsv")

In [6]:
csvFile = spark.read.format("csv")\
    .option("sep", "\t")\
    .option("header", "true")\
    .option("mode", "permissive")\
    .schema(myManualSchema)\
    .load("./data/tmp/my-tsv-file.tsv")

In [7]:
csvFile.show(5)

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|    United States|            Ireland|  264|
|    United States|              India|   69|
|            Egypt|      United States|   24|
|Equatorial Guinea|      United States|    1|
|    United States|          Singapore|   25|
+-----------------+-------------------+-----+
only showing top 5 rows



In [8]:
!ls ./data/tmp/my-tsv-file.tsv

part-00000-8db95157-5f4e-41b4-a729-7c607ff20da4-c000.csv  _SUCCESS


## 9.3 JSON
+ 스파크에서는 JSON 파일을 사용할 때 중로 구분된 JSON을 기본적으로 사용
+ MultiLine 옵션을 사용해 줄로 구분된 방식과 여러 줄로 구성된 방식을 선택적으로 사용할 수 있음

### 9.3.2 JSON 읽기

In [9]:
spark.read.format("json").option("mode", "FAILFAST").option("inferSchema", "true")\
    .load("./data/flight-data/json/2010-summary.json").show(5)

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|    United States|            Romania|    1|
|    United States|            Ireland|  264|
|    United States|              India|   69|
|            Egypt|      United States|   24|
|Equatorial Guinea|      United States|    1|
+-----------------+-------------------+-----+
only showing top 5 rows



### 9.3.3 JSON 쓰기
+ 파티션당 하나의 파일을 만들며 전체 DataFrame을 단일 폴더에 저장
+ JSON 객체는 한 줄에 하나씩 기록됨

In [10]:
csvFile.write.format("json").mode("overwrite")\
    .option("mode", "append")\
    .save("./data/tmp/my-json-file.json")

## 9.4 파케이
+ 전체 파일을 읽는 대신 개별 컬럼을 읽을 수 있으며 컬럼 기반의 압축 기능을 제공
+ 아파치 스파크와 잘 호환되기 때문에 기본 파일 포맷이 됨
+ 복합 데이터 타입을 지원
+ ORC 파일도 파케이와 유사하나 Hive에 최적화 됨

### 9.4.1 파케이 파일 읽기
+ 스키마가 파일 자체에 내장되어 있음

In [11]:
parquetFile = spark.read.format("parquet").load("./data/flight-data/parquet/2010-summary.parquet/")

In [19]:
parquetFile.write.format("parquet").mode("overwrite")\
    .save("./data/tmp/my-parquet-file.parquet")